In [1]:
%matplotlib inline
import os
import numpy as np
import scipy as sp
import datetime
import matplotlib.pyplot as plt
from scipy.integrate import odeint 
import pandas as pd
import scipy as sp 


In [13]:
df_snip=pd.read_csv('../data/snippets/snippets_as_list.tsv',sep='\t')
df_snip=df_snip.loc[:, ~df_snip.columns.str.contains('^Unnamed')]
df_snip

,claimID,snip_id,title,snippet,snippet_url
0,abbc-00001,"[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Record numbers march down Brisbane city to p...","[""Sep 4, 2018 ... THOUSANDS of people joined t...",['http://catholicleader.com.au/news/record-num...
1,abbc-00002,"[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Full text: Senator Fraser Anning's maiden sp...","[""Aug 15, 2018 ... Katter's Australia Party se...",['https://www.sbs.com.au/news/full-text-senato...
2,abbc-00003,"[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Australian senator calls for 'final solution...","[""Aug 14, 2018 ... Crossbencher Fraser Anning ...",['https://www.theguardian.com/australia-news/2...
3,abbc-00004,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",['Fact check: Are Muslims in NSW and Victoria ...,"[""Aug 21, 2018 ... In his headline-grabbing fi...",['https://www.msn.com/en-au/news/australia/fac...
4,abbc-00005,"[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Are more than half of Australia's working-ag...","['Jun 7, 2018 ... Independent Senator Fraser A...",['https://www.youtube.com/watch?v=6bO_YSmTrSw'...
...,...,...,...,...,...
30095,wast-00223,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Democrats' still-misleading claim comparing ...","['Jul 8, 2016 ... She released more emails and...",['https://www.washingtonpost.com/news/fact-che...
30096,wast-00224,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Donald Trump praises Saddam Hussein for kill...","[""Jul 5, 2016 ... Right? He was a bad guy, rea...",['https://www.washingtonpost.com/news/post-pol...
30097,wast-00225,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",['Ted Cruz on Stands By Calling Mitch McConnel...,"['Apr 14, 2016 ... ""Every word I said there is...",['https://www.realclearpolitics.com/video/2016...
30098,wast-00226,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[""North Carolina governor's misleading claim a...","[""Apr 18, 2016 ... To that end, today I have s...",['https://www.washingtonpost.com/news/fact-che...


In [18]:
def clean(x):
    x = x.replace('http://','').replace('https://','').replace('www.','')
    string = x.split('/')

    return string[0]

df_snip['snippet_url'] = df_snip.apply(lambda x: clean(x.snippet_url),axis=1)

In [19]:
df_snip

,claimID,snip_id,title,snippet,snippet_url
0,abbc-00001,"[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Record numbers march down Brisbane city to p...","[""Sep 4, 2018 ... THOUSANDS of people joined t...",['catholicleader.com.au
1,abbc-00002,"[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Full text: Senator Fraser Anning's maiden sp...","[""Aug 15, 2018 ... Katter's Australia Party se...",['sbs.com.au
2,abbc-00003,"[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Australian senator calls for 'final solution...","[""Aug 14, 2018 ... Crossbencher Fraser Anning ...",['theguardian.com
3,abbc-00004,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",['Fact check: Are Muslims in NSW and Victoria ...,"[""Aug 21, 2018 ... In his headline-grabbing fi...",['msn.com
4,abbc-00005,"[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Are more than half of Australia's working-ag...","['Jun 7, 2018 ... Independent Senator Fraser A...",['youtube.com
...,...,...,...,...,...
30095,wast-00223,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Democrats' still-misleading claim comparing ...","['Jul 8, 2016 ... She released more emails and...",['washingtonpost.com
30096,wast-00224,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Donald Trump praises Saddam Hussein for kill...","[""Jul 5, 2016 ... Right? He was a bad guy, rea...",['washingtonpost.com
30097,wast-00225,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",['Ted Cruz on Stands By Calling Mitch McConnel...,"['Apr 14, 2016 ... ""Every word I said there is...",['realclearpolitics.com
30098,wast-00226,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[""North Carolina governor's misleading claim a...","[""Apr 18, 2016 ... To that end, today I have s...",['washingtonpost.com


In [3]:
h1 = ['claimID', 'claim', 'label', 'claimURL', 'reason', 'categories', 'speaker', 'checker', 'tags', 'articleTitle', 'publishDate', 'claimDate', 'entities']

df_train = pd.read_csv('../data/train.tsv',sep='\t',names=h1,header=None)
df_train

,claimID,claim,label,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities
0,pomt-03627,Six out of 10 of the highest unemployment rate...,half-true,/ohio/statements/2013/may/06/chris-redfern/ohi...,When a couple of Statehouse Republicans prepar...,None,Chris Redfern,None,None,None,2013-05-06T06:00:00,2013-04-30,['None']
1,pomt-09611,No Democratic campaign for (Fla.) governor has...,true,/florida/statements/2010/jan/15/alex-sink/flor...,Florida's leading Republican candidate for gov...,None,Alex Sink,None,None,None,2010-01-15T13:59:00,2010-01-06,['None']
2,tron-00214,Forward an email for Jasmine,fiction!,https://www.truthorfiction.com/jasmine/,None,9-11-attack,None,None,None,Forward an email for Jasmine,"Mar 17, 2015",None,['None']
3,snes-04484,Pope Francis endorsed Donald Trump for president.,false,https://www.snopes.com/fact-check/pope-francis...,None,Junk News,None,Dan Evon,None,"Pope Francis Shocks World, Endorses Donald Tru...",10 July 2016,None,['None']
4,pomt-06704,Says Ron Paul insisted FEMA should be shut down.,true,/texas/statements/2011/sep/03/maureen-dowd/mau...,Commenting on the federal response to Hurrican...,None,Maureen Dowd,None,None,None,2011-09-03T06:00:00,2011-08-30,['None']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27866,snes-00146,Did Google Fail to Promote President Trump’s S...,false,https://www.snopes.com/fact-check/google-sotu-...,None,Politics,None,Dan Evon,None,Did Google Fail to Promote President Trump’s S...,30 August 2018,None,['None']
27867,abbc-00012,Federal MP Bob Katter's response to a question...,in-the-red,http://www.abc.net.au/news/2017-11-30/fact-che...,Mr Katter is wrong. Experts told Fact Check th...,"['crocodile', 'bob-katter', 'qld', 'brisbane-4...",None,None,"['crocodile', 'bob-katter', 'qld', 'brisbane-4...",Fact check: Is a person torn to pieces by a cr...,"Thu 12 Jul 2018, 2:14am",None,"['Bob_Katter,_Sr.']"
27868,pomt-05523,"Sen. Sherrod Brown’s vote helped pass ""Barack ...",pants on fire!,/ohio/statements/2012/apr/12/national-republic...,With Congress out for its Easter-Passover brea...,None,National Republican Senatorial Committee,None,None,None,2012-04-12T06:00:00,2012-04-04,"['Ohio', 'Barack_Obama']"
27869,pomt-10920,Even the CBO numbers show now that the entire ...,false,/truth-o-meter/statements/2018/jul/31/larry-ku...,With the economy growing at 4.1 percent in the...,None,Larry Kudlow,None,None,None,2018-07-31T07:00:00,2018-07-29,['Congressional_Budget_Office']


In [14]:
h2 = ['claimID', 'claim', 'claimURL',' reason', 'categories', 'speaker', 'checker',' tags', 'articleTitle', 'publishDate', 'claimDate', 'entities']

df_test = pd.read_csv('../data/test.tsv',sep='\t',names=h2,header=None)
df_test

,claimID,claim,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities
0,pomt-14759,"In 2000, ""I wrote about Osama bin Laden, ‘We’v...",/virginia/statements/2015/dec/11/donald-trump/...,Almost two years before the Sept. 11 terror at...,None,Donald Trump,None,None,None,2015-12-11T10:44:11,2015-12-02,['Osama_bin_Laden']
1,pomt-02596,"Says in 2000, Fox News broke the story of Geor...",/punditfact/statements/2014/jan/27/ann-coulter...,"In the world of cable television news, most pe...",None,Ann Coulter,None,None,None,2014-01-27T16:20:31,2014-01-22,"['Fox_News_Channel', 'George_W._Bush']"
2,snes-00569,A video shows a powerful jet of water flipping...,https://www.snopes.com/fact-check/child-flippe...,None,Fauxtography,None,Dan Evon,None,Does This Video Show a Child Flipped by a Foun...,21 May 2018,None,['None']
3,pomt-14237,My numbers are better right now than Ronald Re...,/truth-o-meter/statements/2016/apr/14/donald-t...,"During a rally in Rome, N.Y., Republican presi...",None,Donald Trump,None,None,None,2016-04-14T10:05:57,2016-04-11,"['Jimmy_Carter', 'Ronald_Reagan']"
4,snes-05947,The U.S. Congress is considering passage of th...,https://www.snopes.com/fact-check/americans-no...,None,Uncategorized,None,David Mikkelson,None,Americans with No Abilities Act,23 June 2006,None,"['United_States_Congress', 'United_States']"
...,...,...,...,...,...,...,...,...,...,...,...,...
3482,vees-00500,Is that so? Duterte-De Lima ‘word war’ on rice...,http://verafiles.org/articles/so-duterte-de-li...,Senator Leila de Lima refuses to back down in ...,None,None,None,"Duterte,de lima,rice smugling",Is that so? Duterte-De Lima ‘word war’ on rice...,"September 02, 2016",None,['None']
3483,snes-03319,Transcript reproduces Ben Stein's television c...,https://www.snopes.com/fact-check/confessions-...,None,Soapbox,None,David Mikkelson,None,Confessions for the Holidays,26 February 2006,None,['Ben_Stein']
3484,pomt-12071,My grandparents were proud immigrants to the U...,/illinois/statements/2017/sep/05/bruce-rauner/...,Bruce Rauner clearly idolized his maternal gra...,None,Bruce Rauner,None,None,None,2017-09-05T05:58:00,2017-08-28,"['United_States', 'Illinois', 'England']"
3485,vees-00229,VERA FILES FACT CHECK: Kuwait DID YIELD to Dut...,http://verafiles.org/articles/vera-files-fact-...,None,None,None,None,false news,VERA FILES FACT CHECK: Kuwait DID NOT YIELD to...,"May 02, 2018",None,['None']


In [22]:
h3 = ['claimID', 'claim', 'label', 'claimURL',' reason', 'categories', 'speaker', 'checker',' tags', 'articleTitle', 'publishDate', 'claimDate', 'entities']

df_dev = pd.read_csv('../data/dev.tsv',sep='\t',names=h3,header=None)
df_dev

,claimID,claim,label,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities
0,tron-00244,Pope Francis: All Dogs Go to Heaven,fiction!,https://www.truthorfiction.com/pope-francis-al...,None,9-11-attack,None,None,None,Pope Francis: All Dogs Go to Heaven – Fiction!,"Mar 17, 2015",None,['None']
1,snes-00534,A photograph shows Subtropical Storm Alberto a...,false,https://www.snopes.com/fact-check/is-this-albe...,None,Fauxtography,None,Dan Evon,None,Is This an Image of Subtropical Storm Alberto ...,29 May 2018,None,"['Pensacola_Beach,_Florida']"
2,snes-05390,A 15-year-old girl named Rosemary Gullett is m...,outdated,https://www.snopes.com/fact-check/missing-chil...,None,Uncategorized,None,David Mikkelson,None,Missing Child: Rosemary Gullett,10 January 2016,None,"['Oregon', 'Portland,_Oregon']"
3,snes-01936,"Walmart placed a sign urging kids to ""own the ...",unproven,https://www.snopes.com/fact-check/walmart-sign...,None,Business,None,Bethania Palma,None,Did Walmart Place a Sign Over a Gun Display Ur...,9 August 2017,None,['None']
4,pomt-15191,Peachtree and Pine is one of the leading sites...,mostly true,/georgia/statements/2015/aug/19/kasim-reed/tb-...,Atlanta Mayor Kasim Reed has made no secret of...,None,Kasim Reed,None,None,None,2015-08-19T00:00:00,2015-08-11,['None']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3479,snes-01833,A vacationing couple staying in a foul-smellin...,true,https://www.snopes.com/fact-check/the-bawdy-un...,None,Horrors,None,David Mikkelson,None,The Body Under the Bed,13 June 1999,None,['None']
3480,snes-03364,"The house-flipping couple from the HGTV show ""...",unproven,https://www.snopes.com/fact-check/flip-or-flop...,None,Entertainment,None,Bethania Palma,None,‘Flip or Flop’ Stars Tarek and Christina El Mo...,12 December 2016,None,['None']
3481,thet-00067,No deal for Britain is better than a bad deal,mostly false,https://theferret.scot/brexit-eu-deal-britain/,None,Fact check Politics,"Theresa May, Conservative Party leader.",None,None,Is no Brexit deal with the EU better than a ba...,"June 11, 2017",None,['United_Kingdom']
3482,pomt-10435,We borrow money from the Chinese to buy oil fr...,mostly true,/truth-o-meter/statements/2008/may/12/hillary-...,Speaking to two of voters' biggest economic co...,None,Hillary Clinton,None,None,None,2008-05-12T00:00:00,2008-02-21,"['China', 'Saudi_Arabia']"


In [20]:
df_Train=pd.merge(df_train, df_snip, how='left', left_on='claimID', right_on='claimID')
df_Train

,claimID,claim,label,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities,snip_id,title,snippet,snippet_url
0,pomt-03627,Six out of 10 of the highest unemployment rate...,half-true,/ohio/statements/2013/may/06/chris-redfern/ohi...,When a couple of Statehouse Republicans prepar...,None,Chris Redfern,None,None,None,2013-05-06T06:00:00,2013-04-30,['None'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]",['Ohio Deserves Right-To-Work | Competitive En...,"['May 8, 2013 ... Ron Maag and Kristina Roegne...",['cei.org
1,pomt-09611,No Democratic campaign for (Fla.) governor has...,true,/florida/statements/2010/jan/15/alex-sink/flor...,Florida's leading Republican candidate for gov...,None,Alex Sink,None,None,None,2010-01-15T13:59:00,2010-01-06,['None'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Andrew Gillum's Improbable Run Against Ron D...","['Oct 30, 2018 ... Resources ... From the star...",['usnews.com
2,tron-00214,Forward an email for Jasmine,fiction!,https://www.truthorfiction.com/jasmine/,None,9-11-attack,None,None,None,Forward an email for Jasmine,"Mar 17, 2015",None,['None'],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Jasmine Abdullah's Leadership Should be Upli...","[""Jun 7, 2016 ... That freedom fighter is Jasm...",['forwardtogether.org
3,snes-04484,Pope Francis endorsed Donald Trump for president.,false,https://www.snopes.com/fact-check/pope-francis...,None,Junk News,None,Dan Evon,None,"Pope Francis Shocks World, Endorses Donald Tru...",10 July 2016,None,['None'],"[1, 3, 4, 5, 6, 7, 8, 9, 10]",['Did the Pope Endorse Trump? - FactCheck.org'...,"['Oct 24, 2016 ... Q: Did Pope Francis endorse...",['factcheck.org
4,pomt-06704,Says Ron Paul insisted FEMA should be shut down.,true,/texas/statements/2011/sep/03/maureen-dowd/mau...,Commenting on the federal response to Hurrican...,None,Maureen Dowd,None,None,None,2011-09-03T06:00:00,2011-08-30,['None'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]",['All statements involving Maureen Dowd | Poli...,"[""Says Ron Paul insisted FEMA should be shut d...",['politifact.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27866,snes-00146,Did Google Fail to Promote President Trump’s S...,false,https://www.snopes.com/fact-check/google-sotu-...,None,Politics,None,Dan Evon,None,Did Google Fail to Promote President Trump’s S...,30 August 2018,None,['None'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Google disputes Trump's claim it failed to p...","[""Aug 29, 2018 ... Google on Wednesday dispute...",['foxbusiness.com
27867,abbc-00012,Federal MP Bob Katter's response to a question...,in-the-red,http://www.abc.net.au/news/2017-11-30/fact-che...,Mr Katter is wrong. Experts told Fact Check th...,"['crocodile', 'bob-katter', 'qld', 'brisbane-4...",None,None,"['crocodile', 'bob-katter', 'qld', 'brisbane-4...",Fact check: Is a person torn to pieces by a cr...,"Thu 12 Jul 2018, 2:14am",None,"['Bob_Katter,_Sr.']","[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Bob Katter's Response To A Question On Same-...","[""Nov 19, 2017 ... Bob Katter's Response To A ...",['huffingtonpost.com.au
27868,pomt-05523,"Sen. Sherrod Brown’s vote helped pass ""Barack ...",pants on fire!,/ohio/statements/2012/apr/12/national-republic...,With Congress out for its Easter-Passover brea...,None,National Republican Senatorial Committee,None,None,None,2012-04-12T06:00:00,2012-04-04,"['Ohio', 'Barack_Obama']","[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Don't be fooled by Trump's tariffs. He's run...","['Mar 13, 2018 ... President-elect Trump at a ...",['washingtonpost.com
27869,pomt-10920,Even the CBO numbers show now that the entire ...,false,/truth-o-meter/statements/2018/jul/31/larry-ku...,With the economy growing at 4.1 percent in the...,None,Larry Kudlow,None,None,None,2018-07-31T07:00:00,2018-07-29,['Congressional_Budget_Office'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Fact check: CBO didn't say tax cuts were 'vi...","[""Aug 2, 2018 ... Fact check: CBO didn't say t...",['usatoday.com


In [23]:
df_Train.to_csv('../data/snip_train.tsv', sep='\t')

In [21]:
df_Test=pd.merge(df_test, df_snip, how='left', left_on='claimID', right_on='claimID')
df_Test

,claimID,claim,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities,snip_id,title,snippet,snippet_url
0,pomt-14759,"In 2000, ""I wrote about Osama bin Laden, ‘We’v...",/virginia/statements/2015/dec/11/donald-trump/...,Almost two years before the Sept. 11 terror at...,None,Donald Trump,None,None,None,2015-12-11T10:44:11,2015-12-02,['Osama_bin_Laden'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]",['Trump boasts that he pointed out Osama bin L...,"['Nov 19, 2018 ... “Of course we should have c...",['nypost.com
1,pomt-02596,"Says in 2000, Fox News broke the story of Geor...",/punditfact/statements/2014/jan/27/ann-coulter...,"In the world of cable television news, most pe...",None,Ann Coulter,None,None,None,2014-01-27T16:20:31,2014-01-22,"['Fox_News_Channel', 'George_W._Bush']","[2, 3, 4, 5, 6, 7, 8, 9, 10]","['Bush Once Pleaded Guilty to DUI', 'The Fox N...","['Nov 3, 2000 ... 3, 2000; 11:27 a.m. EST. MIL...",['washingtonpost.com
2,snes-00569,A video shows a powerful jet of water flipping...,https://www.snopes.com/fact-check/child-flippe...,None,Fauxtography,None,Dan Evon,None,Does This Video Show a Child Flipped by a Foun...,21 May 2018,None,['None'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]",['Girl blasted into air after stepping on foun...,"['Feb 7, 2018 ... Disturbing footage shows a y...",['dailymail.co.uk
3,pomt-14237,My numbers are better right now than Ronald Re...,/truth-o-meter/statements/2016/apr/14/donald-t...,"During a rally in Rome, N.Y., Republican presi...",None,Donald Trump,None,None,None,2016-04-14T10:05:57,2016-04-11,"['Jimmy_Carter', 'Ronald_Reagan']",NaN,NaN,NaN,NaN
4,snes-05947,The U.S. Congress is considering passage of th...,https://www.snopes.com/fact-check/americans-no...,None,Uncategorized,None,David Mikkelson,None,Americans with No Abilities Act,23 June 2006,None,"['United_States_Congress', 'United_States']","[2, 3, 4, 5, 6, 7, 8, 9, 10]","['As Read: Congress Passes ""Americans With No ...","['Congress Passes ""Americans With No Abilities...",['cartalk.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3482,vees-00500,Is that so? Duterte-De Lima ‘word war’ on rice...,http://verafiles.org/articles/so-duterte-de-li...,Senator Leila de Lima refuses to back down in ...,None,None,None,"Duterte,de lima,rice smugling",Is that so? Duterte-De Lima ‘word war’ on rice...,"September 02, 2016",None,['None'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]",['Press Conference of President Rodrigo Roa Du...,"[""Aug 21, 2016 ... And the reason why I could ...",['officialgazette.gov.ph
3483,snes-03319,Transcript reproduces Ben Stein's television c...,https://www.snopes.com/fact-check/confessions-...,None,Soapbox,None,David Mikkelson,None,Confessions for the Holidays,26 February 2006,None,['Ben_Stein'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]","['SteynOnline', 'Soapbox Archives | Snopes.com...","[""Bobby Troup was born one hundred years ago a...",['steynonline.com
3484,pomt-12071,My grandparents were proud immigrants to the U...,/illinois/statements/2017/sep/05/bruce-rauner/...,Bruce Rauner clearly idolized his maternal gra...,None,Bruce Rauner,None,None,None,2017-09-05T05:58:00,2017-08-28,"['United_States', 'Illinois', 'England']","[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Rauner's 'Immigrant' Grandparents Were Actua...","['Sep 6, 2017 ... ""My grandparents were proud ...",['nbcchicago.com
3485,vees-00229,VERA FILES FACT CHECK: Kuwait DID YIELD to Dut...,http://verafiles.org/articles/vera-files-fact-...,None,None,None,None,false news,VERA FILES FACT CHECK: Kuwait DID NOT YIELD to...,"May 02, 2018",None,['None'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]","['About Philippines: Media, Journalism, Bloggi...","[""Vera Files reports cyberattack after publish...",['aboutphilippines.org


In [25]:
df_Test.to_csv('../data/snip_test.tsv', sep='\t')

In [24]:
df_Dev=pd.merge(df_dev, df_snip, how='left', left_on='claimID', right_on='claimID')
df_Dev

,claimID,claim,label,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities,snip_id,title,snippet,snippet_url
0,tron-00244,Pope Francis: All Dogs Go to Heaven,fiction!,https://www.truthorfiction.com/pope-francis-al...,None,9-11-attack,None,None,None,Pope Francis: All Dogs Go to Heaven – Fiction!,"Mar 17, 2015",None,['None'],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","['Pope Francis says dogs can go to heaven', 'D...","['Dec 12, 2014 ... Paradise is open to all of ...",['usatoday.com
1,snes-00534,A photograph shows Subtropical Storm Alberto a...,false,https://www.snopes.com/fact-check/is-this-albe...,None,Fauxtography,None,Dan Evon,None,Is This an Image of Subtropical Storm Alberto ...,29 May 2018,None,"['Pensacola_Beach,_Florida']","[2, 3, 4, 5, 6, 7, 8, 9, 10]",['Alberto nears landfall along Florida Panhand...,"['May 28, 2018 ... The storm was 15 miles west...",['abcnews.go.com
2,snes-05390,A 15-year-old girl named Rosemary Gullett is m...,outdated,https://www.snopes.com/fact-check/missing-chil...,None,Uncategorized,None,David Mikkelson,None,Missing Child: Rosemary Gullett,10 January 2016,None,"['Oregon', 'Portland,_Oregon']","[2, 3, 4, 5, 6, 7, 8, 9, 10]",['Strange photos of missing teen Rosemary Gull...,"[""When her daughter didn't come home one night...",['boredomtherapy.com
3,snes-01936,"Walmart placed a sign urging kids to ""own the ...",unproven,https://www.snopes.com/fact-check/walmart-sign...,None,Business,None,Bethania Palma,None,Did Walmart Place a Sign Over a Gun Display Ur...,9 August 2017,None,['None'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]","[""Walmart apologizes for 'Own the school year ...","['Aug 10, 2017 ... A Walmart gun display urgin...",['usatoday.com
4,pomt-15191,Peachtree and Pine is one of the leading sites...,mostly true,/georgia/statements/2015/aug/19/kasim-reed/tb-...,Atlanta Mayor Kasim Reed has made no secret of...,None,Kasim Reed,None,None,None,2015-08-19T00:00:00,2015-08-11,['None'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]",['Reed wants to shutter Peachtree-Pine homeles...,"['Aug 11, 2015 ... “Peachtree and Pine is one ...",['bizjournals.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3479,snes-01833,A vacationing couple staying in a foul-smellin...,true,https://www.snopes.com/fact-check/the-bawdy-un...,None,Horrors,None,David Mikkelson,None,The Body Under the Bed,13 June 1999,None,['None'],"[2, 3, 4, 5, 6, 7, 8, 9, 10]",['REAL Corpse Found in Mattress…More than Once...,"['Mar 24, 2016 ... Most of us have stayed in h...",['the13thfloor.tv
3480,snes-03364,"The house-flipping couple from the HGTV show ""...",unproven,https://www.snopes.com/fact-check/flip-or-flop...,None,Entertainment,None,Bethania Palma,None,‘Flip or Flop’ Stars Tarek and Christina El Mo...,12 December 2016,None,['None'],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[""'Flip or Flop' Divorce - Why Did Christina &...","['Jan 3, 2019 ... April 2013: The couple\'s HG...",['countryliving.com
3481,thet-00067,No deal for Britain is better than a bad deal,mostly false,https://theferret.scot/brexit-eu-deal-britain/,None,Fact check Politics,"Theresa May, Conservative Party leader.",None,None,Is no Brexit deal with the EU better than a ba...,"June 11, 2017",None,['United_Kingdom'],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",['No deal is often better than a bad deal. Not...,"['Aug 2, 2018 ... “No deal for Britain is bett...",['economist.com
3482,pomt-10435,We borrow money from the Chinese to buy oil fr...,mostly true,/truth-o-meter/statements/2008/may/12/hillary-...,Speaking to two of voters' biggest economic co...,None,Hillary Clinton,None,None,None,2008-05-12T00:00:00,2008-02-21,"['China', 'Saudi_Arabia']",NaN,NaN,NaN,NaN


In [26]:
df_Dev.to_csv('../data/snip_dev.tsv', sep='\t')

In [30]:
#Distribution of labels in df_Train

t=df_Train.label.value_counts()
t_dist=(t/len(df_Train))*100
t_dist

false                    19.866528
mostly true               8.847906
mostly false              8.804851
true                      8.761795
half-true                 8.711564
                           ...    
truth! & disputed!        0.014352
mostly_false              0.014352
grass roots movement!     0.014352
exaggerates               0.014352
cherry picks              0.007176
Name: label, Length: 117, dtype: float64

In [31]:
#Distribution of labels in df_Dev

d=df_Dev.label.value_counts()
d_dist=(d/len(df_Dev))*100
d_dist

false                 19.833525
mostly true            8.869116
mostly false           8.783008
true                   8.754305
half-true              8.725603
                        ...    
understated            0.028703
needs context          0.028703
unobservable           0.028703
truth! & disputed!     0.028703
partially true         0.028703
Name: label, Length: 110, dtype: float64

In [6]:
df_Train.label.value_counts()

false                              5537
mostly true                        2466
mostly false                       2454
true                               2442
half-true                          2428
0                                  1957
pants on fire!                     1287
fiction!                           1246
none                                933
truth!                              621
mixture                             555
promise kept                        441
unproven                            431
facebook scams                      384
1                                   253
true messages                       242
promise broken                      218
compromise                          185
bogus warning                       176
unproven!                           176
in-between                          175
truth! & fiction!                   137
verdict: true                       125
verdict: false                      124
miscaptioned                        122
